Step 1. Identify the files that need to be anonymised. This tool assumes that if there are columns with the same name across files, they refer to the same entity and so it anonymises that information the same way across all files.

In [16]:
import pandas as pd
import random
import json 

files_to_anonymise = ['sample_query_stats.csv','user_activity.csv']

Step 2. Read the file and explore the data in the file to determine what needs to be anonymised

In [25]:
queryStats_dfs = { file_to_anonymise : pd.read_csv(file_to_anonymise) for file_to_anonymise in files_to_anonymise }
    
queryStats_dfs[files_to_anonymise[0]].describe()

,query,newOffer,uniqueOffer,uniqueResult,topicResultsViewed,manualResearchTime,uniqueOffersViewed,uses,airaResultsViewed,uniqueResultViewed,airaSolutions,augementedResearchTime,augmentedSearches,manualSearches,topicSolutions,solutions,TimeSavings,meanManualSolutionTime,meanAugementedSolutionTime
count,1.000000e+04,8166.000000,8236.000000,8236.000000,259.000000,181.000000,211.000000,239.000000,60.000000,60.000000,32.00000,124.000000,64.000000,57.000000,42.000000,22.000000,28.000000,44.000000,29.000000
mean,6.846407e+08,2.529513,2.541646,29.023555,1.494208,482.342790,1.383886,1.476987,1.883333,1.883333,1.53125,508.982935,1.343750,1.666667,1.619048,1.772727,2041.671223,1251.813696,294.581517
std,1.214592e+07,4.726525,4.731641,49.746167,1.847627,2147.129555,0.980537,1.191044,1.508076,1.508076,0.91526,2045.193301,1.011344,1.443376,1.034818,1.109776,5978.409927,4545.002984,557.921368
min,6.144114e+08,0.000000,1.000000,0.000000,1.000000,1.496000,1.000000,1.000000,1.000000,1.000000,1.00000,2.000000,1.000000,1.000000,1.000000,1.000000,-2274.226421,0.000000,2.667000
25%,6.859434e+08,1.000000,1.000000,10.000000,1.000000,44.196000,1.000000,1.000000,1.000000,1.000000,1.00000,8.000000,1.000000,1.000000,1.000000,1.000000,38.091375,62.554010,19.000000
50%,6.875541e+08,1.000000,1.000000,20.000000,1.000000,93.499000,1.000000,1.000000,1.000000,1.000000,1.00000,24.000000,1.000000,1.000000,1.000000,1.000000,231.632372,157.500306,65.000000
75%,6.889446e+08,2.000000,2.000000,20.000000,1.000000,250.151000,1.000000,1.000000,2.000000,2.000000,2.00000,120.837500,1.000000,2.000000,2.000000,2.000000,1214.585490,506.794676,140.638000
max,6.902851e+08,150.000000,150.000000,2245.000000,25.000000,25517.213000,7.000000,9.000000,7.000000,7.000000,4.00000,19524.798000,8.000000,7.000000,4.000000,4.000000,29625.448500,29639.448500,2576.833000


Step 2.1 Identify all the columns in the files

In [27]:
queryStats_dfs
columns_df = {file_to_anonymise : queryStats_df.columns.values.tolist() for file_to_anonymise,queryStats_df in queryStats_dfs.items()}
columns_df

{'sample_query_stats.csv': ['_id',
  'query',
  'user',
  'newOffer',
  'uniqueOffer',
  'uniqueResult',
  'lastUpdated',
  'tech',
  'subtech',
  'topicResultsViewed',
  'manualResearchTime',
  'uniqueOffersViewed',
  'uses',
  'airaResultsViewed',
  'uniqueResultViewed',
  'airaSolutions',
  'augementedResearchTime',
  'augmentedSearches',
  'manualSearches',
  'topicSolutions',
  'solutions',
  'TimeSavings',
  'meanManualSolutionTime',
  'meanAugementedSolutionTime'],
 'user_activity.csv': ['_id',
  'user',
  'dailyactivity',
  'uniqueOffersViewed',
  'uniqueOffersWithSolution',
  'toolComments',
  'toolRating',
  'active',
  'epoch',
  'days',
  'W',
  'c',
  'm',
  'o',
  'meanAugementedSolutionTime',
  'totalAugementedResearchTime',
  'totalManualResearchTime',
  'uniqueOffers',
  'uniqueResult',
  'uses',
  'ratings',
  '2StarAiraResults',
  '3StarAiraResults',
  'topicResultsRated',
  '3StarTopicResults',
  'queries',
  'uniqueQueries',
  'uniqueQueriesWithFeedback',
  'querie

Step 2.2 Look at some of the data in all the columns in the file to determine which ones need anonymisation

In [49]:
queryStats_dfs[files_to_anonymise[0]].head()

,_id,query,user,newOffer,uniqueOffer,uniqueResult,lastUpdated,tech,subtech,topicResultsViewed,...,manualSearches,topicSolutions,solutions,TimeSavings,meanManualSolutionTime,meanAugementedSolutionTime,new_query,new_user,new_tech,new_subtech
0,5c9e5ed7283e6d762fa501af,686433127,jefaguil@cisco.com,1.0,1.0,20.0,2019-03-29T18:07:19.000Z,Security - Network Firewalls and Intrusion Pre...,Sourcefire FirePOWER Services for ASA (SFR Mod...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_1625,user_1847,tech_2,subtech_91
1,5eab0dabb01c737e25728e87,689001333,lrojaslo@cisco.com,1.0,1.0,10.0,2020-04-30T17:40:59.000Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_406,user_1130,tech_26,subtech_109
2,5de0fd778745ddafdf4d695f,688004580,ahmustaf@cisco.com,NaN,NaN,NaN,2019-11-29T11:13:59.000Z,Security - Network Firewalls and Intrusion Pre...,FireSIGHT / FirePOWER - CPU and Memory Utiliza...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_9723,user_654,tech_2,subtech_152
3,5df7cd488745ddafdfb1eac7,688093081,prsivada@cisco.com,1.0,1.0,5.0,2019-12-16T18:30:32.000Z,Security - Network Firewalls and Intrusion Pre...,FireSIGHT / FirePOWER - Access Control Rule (F...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_8392,user_1427,tech_2,subtech_135
4,5cf12b26283e6d762f61f01e,681572296,psotorom@cisco.com,1.0,1.0,20.0,2019-05-31T13:24:54.000Z,Security - VPN and Mobility,VPN with Routers (IPsec),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_3245,user_1518,tech_4,subtech_190


In [29]:
queryStats_dfs[files_to_anonymise[1]].head()

,_id,user,dailyactivity,uniqueOffersViewed,uniqueOffersWithSolution,toolComments,toolRating,active,epoch,days,...,3StarTopicResultsWithoutContext,2StarTopicResultsWithoutContext,numberOfTopicResultTotal,totalTopicResultTotalTime,avgTopicLoadTime,problemsClassified,comments,topicComments,topicProblemsClassified,sawSimilarResults
0,593453ec6bab086cdb55e2d7,nehnaik@cisco.com,NaN,20,8,NaN,NaN,False,472,8,...,0.0,0.0,NaN,NaN,NaN,NaN,132.0,NaN,NaN,NaN
1,593458976bab086cdb55e2d9,namishar@cisco.com,False,157,21,NaN,NaN,True,1254,54,...,7.0,0.0,300.0,5309367.0,17697.89,13.0,115.0,NaN,NaN,NaN
2,59345acd6bab086cdb55e2db,josemed@cisco.com,NaN,13,0,NaN,10.0,False,472,8,...,0.0,0.0,77.0,257085.0,3338.77,NaN,52.0,NaN,NaN,NaN
3,5934dca16bab086cdb55e2e6,ndhingra@cisco.com,False,33,7,NaN,10.0,True,1253,41,...,0.0,0.0,518.0,1974413.0,3811.61,2.0,88.0,NaN,NaN,NaN
4,593504356bab086cdb55e2ed,mgrzesia@cisco.com,False,70,9,NaN,5.0,True,1253,50,...,4.0,1.0,369.0,3141669.0,8514.01,28.0,225.0,NaN,1.0,NaN


Step 3. Define the columns that have data that needs to be anonymised or obfuscated and then map the values in those columns to randomly generated data but store those values in a new column to prevent messing up the existing data

In [37]:
columns_to_obf = [['query','user','tech','subtech'],['user','W','o','m']]
columns_to_obf_dfs = dict(zip(files_to_anonymise, columns_to_obf))
replace = dict(zip([ item for elem in columns_to_obf for item in elem],[{} for elem in columns_to_obf for item in elem]))

for file_to_anonymise in files_to_anonymise:
    for col in columns_to_obf_dfs[file_to_anonymise]:
        values = list(queryStats_dfs[file_to_anonymise][col].unique())
        if replace[col] != {}:
            values.extend(replace[col].keys())
        rand_values = [col+'_'+str(x) for x in random.sample(range(1, len(values)+1), len(values))]
        replace[col].update(dict(zip(values,rand_values)))
        
for file_to_anonymise in files_to_anonymise:
    for col in columns_to_obf_dfs[file_to_anonymise]:
        queryStats_dfs[file_to_anonymise]['new_'+col] = queryStats_dfs[file_to_anonymise][col].map(replace[col])

Step 4. Validate the anonymisation process worked correctly by exploring the data with the newly created columns

In [39]:
queryStats_dfs[files_to_anonymise[0]].head()

,_id,query,user,newOffer,uniqueOffer,uniqueResult,lastUpdated,tech,subtech,topicResultsViewed,...,manualSearches,topicSolutions,solutions,TimeSavings,meanManualSolutionTime,meanAugementedSolutionTime,new_query,new_user,new_tech,new_subtech
0,5c9e5ed7283e6d762fa501af,686433127,jefaguil@cisco.com,1.0,1.0,20.0,2019-03-29T18:07:19.000Z,Security - Network Firewalls and Intrusion Pre...,Sourcefire FirePOWER Services for ASA (SFR Mod...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_1625,user_1847,tech_2,subtech_91
1,5eab0dabb01c737e25728e87,689001333,lrojaslo@cisco.com,1.0,1.0,10.0,2020-04-30T17:40:59.000Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_406,user_1130,tech_26,subtech_109
2,5de0fd778745ddafdf4d695f,688004580,ahmustaf@cisco.com,NaN,NaN,NaN,2019-11-29T11:13:59.000Z,Security - Network Firewalls and Intrusion Pre...,FireSIGHT / FirePOWER - CPU and Memory Utiliza...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_9723,user_654,tech_2,subtech_152
3,5df7cd488745ddafdfb1eac7,688093081,prsivada@cisco.com,1.0,1.0,5.0,2019-12-16T18:30:32.000Z,Security - Network Firewalls and Intrusion Pre...,FireSIGHT / FirePOWER - Access Control Rule (F...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_8392,user_1427,tech_2,subtech_135
4,5cf12b26283e6d762f61f01e,681572296,psotorom@cisco.com,1.0,1.0,20.0,2019-05-31T13:24:54.000Z,Security - VPN and Mobility,VPN with Routers (IPsec),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_3245,user_1518,tech_4,subtech_190


In [ ]:
queryStats_dfs[files_to_anonymise[1]].head()

Step 4.1 Pick some random values from each column that needed to be anonymised and check that they were mapped to the same random value throughout the file

In [42]:
queryStats_df = queryStats_dfs[files_to_anonymise[0]]
for col in columns_to_obf_dfs[files_to_anonymise[0]]:
    print("Validating "+col+"\n----------------------------\n")
    value = random.choice(queryStats_df[col].unique())
    print(queryStats_df[queryStats_df[col] == ][[col, "new_"+col]])
    print("\n----------------------------\n")

Validating query
----------------------------

          query   new_query
9640  687704021  query_6347

----------------------------

Validating user
----------------------------

                    user   new_user
4232  sumesinh@cisco.com  user_1919
4254  sumesinh@cisco.com  user_1919
8155  sumesinh@cisco.com  user_1919
9186  sumesinh@cisco.com  user_1919

----------------------------

Validating tech
----------------------------

                                                   tech new_tech
5     Data Center Computing - (UCS Blade and Rack Mo...  tech_19
17    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
26    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
39    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
40    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
50    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
55    Data Center Computing - (UCS Blade and Rack Mo...  tech_19
58    Data Center Computing - (UCS Blade an

Step 4.2 Pick some random values from each newly created column and check that those values all map to the same value in the corresponding column to be anonymised

In [43]:
for col in columns_to_obf_dfs[files_to_anonymise[0]]:
    print("Validating new_"+col+"\n----------------------------\n")
    print(queryStats_df[queryStats_df["new_"+col] == random.choice(queryStats_df["new_"+col].unique())][[col, "new_"+col]])
    print("\n----------------------------\n")

Validating new_query
----------------------------

          query   new_query
5576  687005997  query_6200

----------------------------

Validating new_user
----------------------------

                    user   new_user
3328  rborreca@cisco.com  user_2952

----------------------------

Validating new_tech
----------------------------

                      tech new_tech
7826  Security: Management  tech_25

----------------------------

Validating new_subtech
----------------------------

                                                subtech  new_subtech
9218  UCS Virtualization and Operating Systems - Red...  subtech_184

----------------------------



Step 5. If every thing looks in order then delete the columns with the data to be anonymised and rename the newly created columns with their old names and store it in a new dataframe

In [45]:
obf_queryStats_dfs = {}
for file_to_anonymise in files_to_anonymise:
    queryStats_df = queryStats_dfs[file_to_anonymise]
    columns_to_obf = columns_to_obf_dfs[file_to_anonymise]
    obf_queryStats_df = queryStats_df.drop(columns_to_obf, axis=1)
    obf_queryStats_df.rename(columns = dict(zip(['new_'+ col for col in columns_to_obf],columns_to_obf)), inplace = True)
    print("Validating colums in "+file_to_anonymise+"\n----------------------------\n")
    print(*obf_queryStats_df.columns.values.tolist(),sep='\n')
    print("\n----------------------------\n")
    obf_queryStats_dfs[file_to_anonymise] = obf_queryStats_df

Validating colums in sample_query_stats.csv
----------------------------

_id
newOffer
uniqueOffer
uniqueResult
lastUpdated
topicResultsViewed
manualResearchTime
uniqueOffersViewed
uses
airaResultsViewed
uniqueResultViewed
airaSolutions
augementedResearchTime
augmentedSearches
manualSearches
topicSolutions
solutions
TimeSavings
meanManualSolutionTime
meanAugementedSolutionTime
query
user
tech
subtech

----------------------------

Validating colums in user_activity.csv
----------------------------

_id
dailyactivity
uniqueOffersViewed
uniqueOffersWithSolution
toolComments
toolRating
active
epoch
days
c
meanAugementedSolutionTime
totalAugementedResearchTime
totalManualResearchTime
uniqueOffers
uniqueResult
uses
ratings
2StarAiraResults
3StarAiraResults
topicResultsRated
3StarTopicResults
queries
uniqueQueries
uniqueQueriesWithFeedback
queriesWithSolution
airaResultViewedNotRated
topicResultViewedNotRated
rank3
rank2
rankAvg
searchContexts
uniqueTopicResults
uniqueTopicResultsWithoutCont

In [47]:
obf_queryStats_dfs[files_to_anonymise[0]].head()

,_id,newOffer,uniqueOffer,uniqueResult,lastUpdated,topicResultsViewed,manualResearchTime,uniqueOffersViewed,uses,airaResultsViewed,...,manualSearches,topicSolutions,solutions,TimeSavings,meanManualSolutionTime,meanAugementedSolutionTime,query,user,tech,subtech
0,5c9e5ed7283e6d762fa501af,1.0,1.0,20.0,2019-03-29T18:07:19.000Z,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_1625,user_1847,tech_2,subtech_91
1,5eab0dabb01c737e25728e87,1.0,1.0,10.0,2020-04-30T17:40:59.000Z,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_406,user_1130,tech_26,subtech_109
2,5de0fd778745ddafdf4d695f,NaN,NaN,NaN,2019-11-29T11:13:59.000Z,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_9723,user_654,tech_2,subtech_152
3,5df7cd488745ddafdfb1eac7,1.0,1.0,5.0,2019-12-16T18:30:32.000Z,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_8392,user_1427,tech_2,subtech_135
4,5cf12b26283e6d762f61f01e,1.0,1.0,20.0,2019-05-31T13:24:54.000Z,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,query_3245,user_1518,tech_4,subtech_190


Step 6. Write this newly created dataframe to a new csv file

In [48]:
for file_to_anonymise in files_to_anonymise:
    obf_queryStats_dfs[file_to_anonymise].to_csv(r'anon_'+file_to_anonymise, index = False)